In [1]:
import os

In [2]:
%pwd

'e:\\MLProject\\chicken-disease\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\MLProject\\chicken-disease'

In [5]:
import tensorflow as tf

In [6]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class Evaluation_config:
    path_of_model:Path
    training_data:Path
    all_params:dict
    params_image_size:list
    params_batch_size: int
    

In [13]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self)->Evaluation_config:
        eval_config=Evaluation_config(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE


        )

        return eval_config

In [15]:
from urllib.parse import urlparse
from pathlib import Path

In [16]:
class Evaluation:
    def __init__(self, config:Evaluation_config):
        self.config=config

    def _valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self._valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs

        )

    @staticmethod
    def load_model(path: Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self._valid_generator)
        
    def save_score(self):
        score={"loss":self.score[0], "accuracy":self.score[1]}
        save_json(path=Path("score.json"), data=score)

        

In [17]:
try:
    config=ConfigurationManager()
    val_config=config.get_validation_config()
    evaluation=Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

Found 116 images belonging to 2 classes.
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.5000 - loss: 17.5732
